# Setup

In [15]:
# Extract and build the benchmark
!make -C io

!mkdir -p /data/lab1
!io/io-static -c lab1/iofile

Benchmark configuration:
  buffersize: 16384
  totalsize: 16777216
  blockcount: 1024
  operation: create
  path: lab1/iofile
  time: 2.344273708
speed: 6988.95 KBytes/sec


# Reads

## Constant IO/Varying Buffer (with cache, static/dynamic)

* Total size: 16MB (default)
* Buffer: varies from 512 to 512 * 2^16
* (io.c modified to output kbytes/sec in same format as other params)
* Run multiple times then average
* Prevent warmup effect

In [16]:
# Imports
import time

# Setup
BUF_START = 512
BUFFER_SIZES = [512 * 2 ** exp for exp in range(0, 16)]
FILES = ["lab1/iofile", "/dev/zero"]
EXECUTABLES = ["io/io-static", "io/io-dynamic"]
IO_SIZE = BUFFER_SIZES[-1] # Keep constant at default 16MB (last value of BUFFER_SIZES)
RUNS = range(1, 16) # 15 runs

def strs_to_tup(strs):
    if len(strs) == 1:
        return (strs[0].strip(),None)
    else:
        return (strs[0].strip(),strs[1].strip())

def parse(cmd_out):
    tups = map(strs_to_tup, [string.split(":") for string in cmd_out])
    return dict(tups)

# The meat

outputs = {}
for exe in EXECUTABLES:
    print("\t ==> Running {}".format(exe))
    runs = []
    for filename in FILES:
        print("\t\t ==> Reading from {}".format(filename))
        for buffer_size in BUFFER_SIZES:
            results = []
            for i in RUNS:
                out = !{exe} -v -r -b {str(buffer_size)} -t {str(IO_SIZE)} {filename}
                parsed = parse(out)
                results.append(parsed)
                time.sleep(2) # Sleep for 2 seconds to prevent warmup effect

            runs.append(results)
            times = [float(item['time']) for item in results]
            speeds = [float(item['speed'].split(' ')[0]) for item in results] # Need to split by ' ' because output speed has units attached to it (see io.c)
            avg_time = sum(times) / len(RUNS)
            avg_speed = sum(speeds) / len(RUNS)
            buf_sz = int(results[0]['buffersize'])
            tot_sz = int(results[0]['totalsize'])
            print("\t\t\t{0:2} bytes {1:2} bytes ({2:.2f} KBytes/sec): {3:.6f}s".format(buf_sz, tot_sz, avg_speed, avg_time))
    outputs.update({exe:runs})

	 ==> Running io/io-static
		 ==> Reading from lab1/iofile
			512 bytes 16777216 bytes (39589.28): 0.413849s
			1024 bytes 16777216 bytes (68180.77): 0.240302s
			2048 bytes 16777216 bytes (101903.25): 0.160780s
			4096 bytes 16777216 bytes (139560.34): 0.117397s
			8192 bytes 16777216 bytes (159344.82): 0.102821s
			16384 bytes 16777216 bytes (172057.69): 0.095224s
			32768 bytes 16777216 bytes (177517.50): 0.092295s
			65536 bytes 16777216 bytes (175863.02): 0.093163s
			131072 bytes 16777216 bytes (172307.55): 0.095086s
			262144 bytes 16777216 bytes (171258.49): 0.095668s
			524288 bytes 16777216 bytes (170876.27): 0.095882s
			1048576 bytes 16777216 bytes (170665.70): 0.096001s
			2097152 bytes 16777216 bytes (167278.69): 0.097944s
			4194304 bytes 16777216 bytes (161741.60): 0.101297s
			8388608 bytes 16777216 bytes (75603.23): 0.216710s
			16777216 bytes 16777216 bytes (128741.09): 0.127263s
		 ==> Reading from /dev/zero
			512 bytes 16777216 bytes (249226.29): 0.065739s
			1024

{'io/io-dynamic': [[{'blockcount': '32768', 'totalsize': '16777216', 'Benchmark configuration': '', 'time': '0.402354792', 'path': 'lab1/iofile', 'operation': 'read', 'buffersize': '512', 'speed': '40720.28 KBytes/sec'}], [{'blockcount': '16384', 'totalsize': '16777216', 'Benchmark configuration': '', 'time': '0.236671167', 'path': 'lab1/iofile', 'operation': 'read', 'buffersize': '1024', 'speed': '69226.85 KBytes/sec'}], [{'blockcount': '8192', 'totalsize': '16777216', 'Benchmark configuration': '', 'time': '0.161311791', 'path': 'lab1/iofile', 'operation': 'read', 'buffersize': '2048', 'speed': '101567.28 KBytes/sec'}], [{'blockcount': '4096', 'totalsize': '16777216', 'Benchmark configuration': '', 'time': '0.119176666', 'path': 'lab1/iofile', 'operation': 'read', 'buffersize': '4096', 'speed': '137476.58 KBytes/sec'}], [{'blockcount': '2048', 'totalsize': '16777216', 'Benchmark configuration': '', 'time': '0.103006709', 'path': 'lab1/iofile', 'operation': 'read', 'buffersize': '8192